<a href="https://colab.research.google.com/github/AyushiRwt20/Email-Spam-Detection-/blob/main/Spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**E-MAIL SPAM DETECTION**

#**Importing** **Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score,f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


#**Importing** **Data** **from** **Drive**

In [ ]:
import pandas as pd
from google.colab import drive
print("importing csv file from drive")
drive.mount('/content/drive')
path="/content/drive/MyDrive/mail_data.csv"
df=pd.read_csv(path)
ch=input("want to display the dataframe?( Y/N):")
if (ch=="Y" or ch=="y"):
  print(df.head(4))
else:
  print("OK continuing the programme.")
print("-*-"*20)

importing csv file from drive
Mounted at /content/drive
want to display the dataframe?( Y/N):n
OK continuing the programme.
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-


# **Data** **Cleaning**

In [ ]:
print(df.shape)

df.describe()
df.isnull().sum()
print("counting  non-null values..")
print("total non-null values in columns:-",df.count(1))
cols = list(df.columns)
print(cols)

(5572, 2)
counting  non-null values..
total non-null values in columns:- 0       2
1       2
2       2
3       2
4       2
       ..
5567    2
5568    2
5569    2
5570    2
5571    2
Length: 5572, dtype: int64
['Category', 'Message']


#**Logistic** **Regression**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
df.shape


(5572, 2)

In [ ]:
df.loc[df['Category']=='spam','Category' ,] = 0
df.loc[df['Category']=='ham','Category' ,] = 1

In [ ]:
X = df['Message']
Y = df['Category']

In [ ]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [ ]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2 , random_state = 3)

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


In [ ]:
print(Y.shape)
print(Y_train.shape)
print(Y_test.shape)

(5572,)
(4457,)
(1115,)


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

Vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, min_df=1)
feature_extraction = Vectorizer.fit(X_train)
X_train_features = feature_extraction.transform(X_train)
X_test_features = feature_extraction.transform(X_test)

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [ ]:
print(X_train)

3075                  Don know. I did't msg him recently.
1787    Do you know why god created gap between your f...
1614                         Thnx dude. u guys out 2nite?
4304                                      Yup i'm free...
3266    44 7732584351, Do you want a New Nokia 3510i c...
                              ...                        
789     5 Free Top Polyphonic Tones call 087018728737,...
968     What do u want when i come back?.a beautiful n...
1667    Guess who spent all last night phasing in and ...
3321    Eh sorry leh... I din c ur msg. Not sad alread...
1688    Free Top ringtone -sub to weekly ringtone-get ...
Name: Message, Length: 4457, dtype: object


In [ ]:
print(X_train_features)

  (0, 5413)	0.6198254967574347
  (0, 4456)	0.4168658090846482
  (0, 3811)	0.34780165336891333
  (0, 2329)	0.38783870336935383
  (0, 2224)	0.413103377943378
  (1, 4080)	0.18880584110891163
  (1, 3811)	0.17419952275504033
  (1, 3325)	0.31610586766078863
  (1, 3185)	0.29694482957694585
  (1, 3046)	0.2503712792613518
  (1, 2957)	0.3398297002864083
  (1, 2956)	0.33036995955537024
  (1, 2758)	0.3226407885943799
  (1, 2746)	0.3398297002864083
  (1, 1991)	0.33036995955537024
  (1, 1839)	0.2784903590561455
  (1, 918)	0.22871581159877646
  (2, 6601)	0.6056811524587518
  (2, 3156)	0.4107239318312698
  (2, 2404)	0.45287711070606745
  (2, 407)	0.509272536051008
  (3, 7414)	0.8100020912469564
  (3, 2870)	0.5864269879324768
  (4, 7090)	0.1468655885827845
  (4, 6551)	0.20583643502688098
  :	:
  (4454, 2855)	0.47210665083641806
  (4454, 2246)	0.47210665083641806
  (4455, 7113)	0.30536590342067704
  (4455, 6916)	0.19636985317119715
  (4455, 6810)	0.29731757715898277
  (4455, 6091)	0.23103841516927642
  

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train_features,Y_train)

LogisticRegression()

In [ ]:
prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train , prediction_on_training_data)

In [ ]:
print('Acc on training data : ' ,accuracy_on_training_data)

Acc on training data :  0.9670181736594121


In [ ]:
prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test , prediction_on_test_data)
print('Acc on training data : ' ,accuracy_on_training_data)

Acc on training data :  0.9670181736594121


In [ ]:
input_your_mail = ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
input_data_features = feature_extraction.transform(input_your_mail)
prediction = model.predict(input_data_features)
print(prediction)
if(prediction[0] == 1):
    print('HAM MAIL')
else :
    print('SPAM MAIL')


[0]
SPAM MAIL


# KNN

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
# Replace 'k_value' with the desired number of neighbors (e.g., 5)
k_value = 5
knn_classifier = KNeighborsClassifier(n_neighbors=k_value)
knn_classifier.fit(X_train, Y_train)


KNeighborsClassifier()

In [ ]:
y_pred = knn_classifier.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8484304932735426


In [ ]:
# Print confusion matrix
conf_matrix = confusion_matrix(Y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[  0 155]
 [ 14 946]]


In [ ]:
# Print classification report
class_report = classification_report(Y_test, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       155
           1       0.86      0.99      0.92       960

    accuracy                           0.85      1115
   macro avg       0.43      0.49      0.46      1115
weighted avg       0.74      0.85      0.79      1115



#**Data** **Visulaization**


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


1. DATA CLEANING

In [ ]:
# basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
#Renaming the Columns
df.rename({'v1':'Type','v2':'SMS'},axis=1,inplace=True)
df

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will ü b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...


In [ ]:
## Removing the Last Three Columns

final_df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=0)
final_df.sample(5)

KeyError: ignored

In [ ]:
## Changing My Target to Labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
final_df['Type'] = le.fit_transform(final_df['Type'])

In [ ]:
final_df.sample(5)

In [ ]:
## Checking Missing Values
final_df.isnull().sum()

In [ ]:
##Check for Duplicated Values
final_df.duplicated().sum()

In [ ]:
##Remove Duplicates
final_df = final_df.drop_duplicates(keep='first')
final_df.head()

In [ ]:
final_df.shape

In [ ]:
final_df.duplicated().sum()

2. EDA

In [ ]:
plt.pie(final_df['Type'].value_counts(),labels=['ham','spam'],autopct='%0.2f%%',explode=[0.1,0])
plt.show()

In [ ]:
# data is imbalaced
## Now we are going to do Analysis on No. of Characters, Words, and Sentences Used in every Message

import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
## Number of Characters
final_df['num_characters'] = final_df['SMS'].apply(len)
final_df.head()

In [ ]:
## Number of Words
final_df['num_words'] = final_df['SMS'].apply(lambda x:len(nltk.word_tokenize(x)))
final_df.head()

In [ ]:
final_df['num_sentences'] = final_df['SMS'].apply(lambda x:len(nltk.sent_tokenize(x)))
final_df.head()

In [ ]:
##ham
final_df[final_df['Type'] == 0][['num_characters','num_words','num_sentences']].describe()

In [ ]:
## spam
final_df[final_df['Type'] == 1][['num_characters','num_words','num_sentences']].describe()

In [ ]:
import seaborn as sns

plt.figure(figsize=(10,8))
sns.histplot(final_df[final_df['Type'] == 0]['num_characters'])
sns.histplot(final_df[final_df['Type'] == 1]['num_characters'],color='red')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.histplot(final_df[final_df['Type'] == 0]['num_words'])
sns.histplot(final_df[final_df['Type'] == 1]['num_words'],color='red')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.histplot(final_df[final_df['Type'] == 0]['num_sentences'])
sns.histplot(final_df[final_df['Type'] == 1]['num_sentences'],color='red')
plt.show()

In [ ]:
sns.pairplot(final_df,hue='Type')
plt.show()

In [ ]:
sns.heatmap(final_df.corr(),annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(x='Type',y='num_characters',data=final_df)

# 3. Data Preprocessing

    1. Lowercase
    2. Tokenization
    3. Removing Special Character
    4. Removing stop words and punctuation
    5. stemming

In [ ]:
# let's do data preprocessing like remove punctucation, stopwords, stemming.
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer

ps = PorterStemmer()

def text_processing(text):
    text = nltk.word_tokenize(text.lower())
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            stemming = ps.stem(i)
            y.append(stemming)

    return " ".join(y)

In [ ]:
text_processing('I Loved the YT Lectures on machine Learning What About You! dacing dance danced')

NameError: ignored

In [ ]:
final_df['transformed_text'] = final_df['SMS'].apply(text_processing)

In [ ]:
final_df.head()

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(width=500,height=500,min_font_size=10,background_color='white')

In [ ]:
## Spam Word Cloud
spam_wc = wc.generate(final_df[final_df['Type'] == 1]['transformed_text'].str.cat(sep = " "))
plt.figure(figsize=(8,5))
plt.imshow(spam_wc)
plt.show()

In [ ]:
## Ham Word Cloud
ham_wc = wc.generate(final_df[final_df['Type'] == 0]['transformed_text'].str.cat(sep = " "))
plt.figure(figsize=(8,5))
plt.imshow(ham_wc)
plt.show()

In [ ]:
## Converting Spam Messages to List of Words
x = final_df[final_df['Type'] == 1]['transformed_text'].str.cat(sep= " ")
spam_msg_word_list = x.split()
len(spam_msg_word_list)

In [ ]:
## Counting Top 30 Words from spam messages
from collections import Counter
spam_top_30words = pd.DataFrame(Counter(spam_msg_word_list).most_common(30))

In [ ]:
## List of the Total Ham Words

ham_msg_list = final_df[final_df['Type'] == 0]['transformed_text'].tolist()
ham_msg_word_list = []

for i in ham_msg_list:
    for j in i.split():
        ham_msg_word_list.append(j)

In [ ]:
## Converting Most Comman 30 WOrds to DataFrame
ham_top_30words = pd.DataFrame(Counter(ham_msg_word_list).most_common(30))
ham_top_30words.head()

#**Naive** **Bayes**


In [ ]:
df.groupby("Category").describe()


spam                                   
           count mean  std  min  25%  50%  75%  max
Category                                           
ham       4825.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
spam       747.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [ ]:
df["spam"]=df["Category"].apply(lambda x: 1 if x=="spam" else 0)

In [ ]:
x_train ,x_test,y_train,y_test=train_test_split(df.Message,df.spam, train_size=0.3)
x_train.describe()

count                       1671
unique                      1619
top       Sorry, I'll call later
freq                           9
Name: Message, dtype: object

In [ ]:

cv=CountVectorizer()
x_train_count=cv.fit_transform(x_train.values)
x_train_count

<1671x4398 sparse matrix of type '<class 'numpy.int64'>'
	with 22377 stored elements in Compressed Sparse Row format>

In [ ]:
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
model=MultinomialNB()
model.fit(x_train_count,y_train)

MultinomialNB()

In [ ]:
email_ham=["hey wanna comeover?"]
email_ham_count=cv.transform(email_ham)
model.predict(email_ham_count)

array([0])

In [ ]:
email_spam=["Congrats you just won 100k!"]
email_spam_count=cv.transform(email_spam)
model.predict(email_spam_count)

array([1])

In [ ]:
emails=["hey wanna comeover?","Congrats you just won 100k!"]
emails_count=cv.transform(emails)
model.predict(emails_count)

array([0, 1])

In [ ]:
x_test_count=cv.transform(x_test)
model.score(x_test_count,y_test)

0.9797487823634965

In [ ]:
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ("vectorizer",CountVectorizer())
    ,("nb",MultinomialNB())])

In [ ]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [ ]:
clf.score(x_test,y_test)

0.9797487823634965

In [ ]:
clf.predict(emails)

array([0, 1])

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sn
y_predicted=model.predict(x_test_count)
cm=confusion_matrix(y_test,y_predicted)
print(cm)
#plt.figure(figsize=(10,7))
#sn.heatmap(cm,annot="True")
#plt.xlabel("truth")
#plt.ylabel("predicted")



[[3353   18]
 [  61  469]]


#**SUPPORT** **VECTOR** **CLASSIFIER**

In [ ]:

X = df['Message']
y = df['Category']
y = y.map({'ham': 0, 'spam': 1})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
new_messages = ["Hello, congratulations! You've won a free trip. Reply now to claim your prize.",
                "Hi, this is a friendly reminder about our upcoming meeting.",
                "URGENT: Your account will be suspended if you don't take action immediately."]

new_messages_transformed = vectorizer.transform(new_messages)
predictions = svc.predict(new_messages_transformed)

In [ ]:
for message, prediction in zip(new_messages, predictions):
    print("Message:", message)
    print("Prediction:", prediction)
    print()

Message: Hello, congratulations! You've won a free trip. Reply now to claim your prize.
Prediction: 1

Message: Hi, this is a friendly reminder about our upcoming meeting.
Prediction: 0

Message: URGENT: Your account will be suspended if you don't take action immediately.
Prediction: 0



**confusion marix**

In [ ]:
y_pred = svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9937219730941704
